## 필요 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import export_graphviz
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

C:\Anaconda3\envs\nlp\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Anaconda3\envs\nlp\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Anaconda3\envs\nlp\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


## 데이터 크롤링

In [6]:
# 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
# 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
# webdriver 설치 위치를 정의한다.
browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')

def crawling(url):
    # 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
    # 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
    # webdriver 설치 위치를 정의한다.
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')
    # 불러오고자하는 url을 .get으로 호출한다.
    browser.get(url)
    
    # page_source파라미터를 이용하면 HTML정보를 가지고 온다.
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')   # 뷰티풀숩으로 HTML을 파싱하고 필요한 데이터 수집
    table_tags = soup.find_all("table")   # find_all함수를 이용하면, TABLE태그로 지정된 곳만 뽑아서, 배열 형태로 저장
    table = table_tags[0]    # html상에서 table 순서, 첫번째 테이블 가지고 와야 함.
    p=parser.make2d(table)

    df_total=pd.DataFrame(p[1:],columns=p[0])    # 데이터 프레임으로 저장

    num = soup.find_all("button")[5].text   # 5번째 button에 페이지수 정보가 있음
    for j in range(1, int(num)):
        btn = browser.find_element_by_class_name('paginationWidget-next')
        btn.click()     # 버튼 클릭
        time.sleep(5)

        # 2페이지 이후 데이터 병합
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table_tags = soup.find_all("table")
        table = table_tags[0]
        p = parser.make2d(table)

        df = pd.DataFrame(p[1:], columns=p[0])
        df_total = pd.concat([df_total, df], 0)
        
    return df_total
        # 엑셀로 다운로드
        # df_total.to_excel("test.xlsx")

In [7]:
list_url = ["http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+hitting&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=hitting&page=1&ts=1580532864437&playerType=ALL",
"http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+fielding&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=fielding&page=1&ts=1580539033522&playerType=ALL"]

df_total_hit = crawling(list_url[0])
df_total_field = crawling(list_url[1])

## 전처리 진행
- 불필요한 컬럼 삭제(G, GS) , RK, Team, 빈 컬럼, 등
- hit & field data mergy : key column Player & Team $ Position
- 1B, 2B, 3B, SS 포지션만 추출
- AB 타석수(20), INN(20)가 일정 값 이상의 선수만 대상으로 한다.


In [8]:
# 컬럼 rename
df_total_hit = df_total_hit.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', 
                                            '':'del', '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'AB▼':'AB', 'R▼':'R',
                                            'H▼':'H', '2B▼':'2B', '3B▼':'3B', 'HR▼':'HR',
                                            'RBI▼':'RBI', 'BB▼':'BB', 'SO▼':'SO', 'SB▼':'SB',
                                            'CS▼':'CS', 'AVG▼':'AVG', 'OBP▼':'OBP',
                                            'SLG▼':'SLG', 'OPS▼':'OPS', 'IBB▼':'IBB',
                                            'HBP▼':'HBP', 'SAC▼':'SAC', 'SF▼':'SF', 
                                            'TB▼':'TB', 'XBH▼':'XBH', 'GDP▼':'GDP',
                                            'GO▼':'GO', 'AO▼':'AO', 'GO_AO▼':'GO_AO',
                                            'NP▼':'NP', 'PA▼':'PA'})
df_total_field = df_total_field.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', '':'del',
                                            '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'GS▼':'GS',
                                            'INN▼':'INN', 'TC▼':'TC', 'PO▼':'PO', 'A▼':'A',
                                            'E▼':'E', 'DP▼':'DP', 'SB▼':'SB', 'CS▼':'CS',
                                            'SBPCT▲':'SBPCT', 'PB▼':'PB', 'C_WP▼':'C_WP',
                                            'FPCT▼':'FPCT', 'RF▼':'RF'})

In [9]:
df_total_hit.to_excel("df_total_hit.xlsx")
df_total_hit

,RK,Player,Team,del,del2,Pos,G,AB,R,H,...,SAC,SF,TB,XBH,GDP,GO,AO,GO_AO,NP,PA
0,1,"Alaniz, R",CIN,,595798,P,8,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
1,1,"Brebbia, J",STL,,605154,P,63,1,1,1,...,0,0,1,0,0,0,0,-.--,1,1
2,1,"Drake, O",TB,,543118,P,3,1,0,1,...,0,0,1,0,0,0,0,-.--,4,1
3,1,"Liriano, F",PIT,,434538,P,67,1,0,1,...,0,0,1,0,0,0,0,-.--,15,2
4,1,"Lugo, S",NYM,,607625,P,57,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,984,"Yates, K",SD,,489446,P,59,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
33,984,"Ynoa, G",BAL,,593679,P,4,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
34,984,"Ynoa, H",ATL,,660623,P,2,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
35,984,"Zamora, D",NYM,,623354,P,17,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0


In [10]:
df_total_field.to_excel("df_total_field.xlsx")
df_total_field

,RK,Player,Team,del,del2,Pos,G,GS,INN,TC,...,A,E,DP,SB,CS,SBPCT,PB,C_WP,FPCT,RF
0,1,"Abreu, B",HOU,,650556,P,7,0,8.2,4,...,2,0,0,-,-,-,-,-,1.000,0.57
1,1,"Adames, C",SF,,542436,SS,7,5,46.0,15,...,8,0,2,-,-,-,-,-,1.000,2.14
2,1,"Adams, A",DET,,542866,P,15,0,16.2,4,...,3,0,0,-,-,-,-,-,1.000,0.27
3,1,"Adams, C",NYY,,664856,P,13,0,25.1,2,...,2,0,0,-,-,-,-,-,1.000,0.15
4,1,"Adduci, J",CHC,,451192,RF,1,1,8.0,5,...,0,0,0,-,-,-,-,-,1.000,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1319,"Walker, T",ARI,,592836,P,1,1,1.0,0,...,0,0,0,-,-,-,-,-,-,0.00
1,1319,"Warren, A",SEA,,605521,P,6,0,5.1,0,...,0,0,0,-,-,-,-,-,-,0.00
2,1319,"Williams, A",WSH,,657117,P,2,0,0.1,0,...,0,0,0,-,-,-,-,-,-,0.00
3,1319,"Williams, D",MIL,,642207,P,13,0,13.2,0,...,0,0,0,-,-,-,-,-,-,0.00


In [2]:
df_total_hit_ = pd.read_excel("df_total_hit.xlsx")
df_total_field_ = pd.read_excel("df_total_field.xlsx")

In [3]:
# 데이터 백업

player_hit_data = df_total_hit_
player_field_data = df_total_field_
player_hit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 35 columns):
Unnamed: 0    1287 non-null int64
RK            1287 non-null int64
Player        1287 non-null object
Team          1287 non-null object
del           0 non-null float64
del2          1287 non-null int64
Pos           1287 non-null object
G             1287 non-null int64
AB            1287 non-null int64
R             1287 non-null int64
H             1287 non-null int64
2B            1287 non-null int64
3B            1287 non-null int64
HR            1287 non-null int64
RBI           1287 non-null int64
BB            1287 non-null int64
SO            1287 non-null int64
SB            1287 non-null int64
CS            1287 non-null int64
AVG           1287 non-null object
OBP           1287 non-null object
SLG           1287 non-null object
OPS           1287 non-null float64
IBB           1287 non-null int64
HBP           1287 non-null int64
SAC           1287 non-null int64
SF

In [4]:
# 불필요한 컬럼 삭제

player_hit_data = player_hit_data.drop(['Unnamed: 0', 'RK', 'del', 'del2', 'GO_AO', 'G'], axis=1)
player_field_data = player_field_data.drop(['Unnamed: 0', 'RK', 'del', 'del2', 'G', 'GS', 'SB', 'CS', 'SBPCT','PB','C_WP'], axis=1)

In [5]:
# column Player & Team $ Position 기준 병합

df_total = pd.merge(player_hit_data, player_field_data, on=['Player', 'Team', 'Pos']) # how='outer')
df_total.columns
#df_total.to_excel("test.xlsx")

Index(['Player', 'Team', 'Pos', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
       'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC', 'SF',
       'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN', 'TC', 'PO', 'A', 'E',
       'DP', 'FPCT', 'RF'],
      dtype='object')

In [6]:
# 컬럼 조건문으로 행 추출하기 Position 컬럼에서 "1B", "2B", "3B", "SS"

# 하나의 조건문이라면 "해당 컬럼 == 하나조건"으로 하면 되지만, "해당컬럼.isin([여러조건])"
# df_total_hit_infld = df_total_hit[df_total_hit['Pos'] == "2B"]
df_total_infld = df_total[df_total['Pos'].isin(["1B", "2B", "3B", "SS"])]
df_total_infld
print(df_total_infld.shape)

(288, 37)


In [7]:
# df_total_infld.info()
# df_total_infld.dtypes 데이터 타입만 보려면

### 이렇게 해서 데이터 타입을 바꾸니 문제가 있네
```
# object --> numeric(int or float) 
# 현재 데이터는 전부 object type이여서 수치형 데이터는 pd.to_numeric함수를 이용해 변환
# 데이터프레임에서 컬럼으로 슬라이싱하려면 iloc을 사용
# df_total_infld['G_x'] = pd.to_numeric(df_total_infld['G_x'])
df_total_infld = df_total_infld.apply(pd.to_numeric.iloc, errors='coerce')

# 문자열 타입인데 numeric 타입으로 바뀐것을 원상 복구
# numeric(int or float) --> object
df_total_infld.astype({'Player':'str', 'Team':'str', 'Pos':'str'}).info()
```
- object가 numeric으로 바뀌니깐 NaN값이 되어 버림

- list로 해서 데이터 타입 일괄로 numeric으로 바꾸기
```
col_list = ['AB', 'R', 'H', '2B', '3B', 'HR', 'RBI',
       'BB', 'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC',
       'SF', 'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN',
       'TC', 'PO', 'A', 'E', 'DP', 'FPCT', 'RF']
for i in col_list:
    df_total_infld[i] = pd.to_numeric(df_total_infld[i], errors='coerce')
```

In [8]:
# 타석수와 수비이닝의 최소값 제한

df_total_infld = df_total_infld[df_total_infld['AB'] >= 20]
print(df_total_infld.shape)
df_total_infld = df_total_infld[df_total_infld['INN'] >= 20]
print(df_total_infld.shape)

(274, 37)
(273, 37)


In [9]:
# Null 여부 확인
df_total_infld.isnull().count()

Player    273
Team      273
Pos       273
AB        273
R         273
H         273
2B        273
3B        273
HR        273
RBI       273
BB        273
SO        273
SB        273
CS        273
AVG       273
OBP       273
SLG       273
OPS       273
IBB       273
HBP       273
SAC       273
SF        273
TB        273
XBH       273
GDP       273
GO        273
AO        273
NP        273
PA        273
INN       273
TC        273
PO        273
A         273
E         273
DP        273
FPCT      273
RF        273
dtype: int64

- 특정 컬럼의 null값이 몇개 있는지 확인
```
df_total_infld.iloc[:,3:]
print(df_total_infld['FPCT'].isnull().value_counts())
```

- NaN값을 N으로 변경
```
df_total_infld['FPCT'].fillna('N', inplace=True)
df_total_infld.info()
```

## 데이터 분석하기
- groupby를 이용하여 분석하기


In [10]:
df_total_infld

,Player,Team,Pos,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,AVG,OBP,SLG,OPS,IBB,HBP,SAC,SF,TB,XBH,GDP,GO,AO,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
22,"Kendrick, H",WSH,3B,334,61,115,23,1,17,62,27,49,2,1,.344,.395,.572,0.966,1,4,0,5,191,41,11,93,82,1397,370,601.0,401,308,91,2,35,.995,4.29
23,"Anderson, T",CWS,SS,498,81,167,32,0,18,56,15,109,17,5,.335,.357,.508,0.865,0,3,0,2,253,50,12,131,93,1765,518,1050.0,532,182,324,26,75,.951,4.11
24,"Arraez, L",MIN,2B,326,54,109,20,1,4,28,36,29,2,2,.334,.399,.439,0.838,1,1,0,3,143,25,2,86,105,1486,366,716.0,227,101,121,5,31,.978,2.22
37,"Solano, D",SF,SS,215,27,71,13,1,4,23,10,49,0,1,.330,.360,.456,0.815,0,1,0,2,98,18,4,43,54,903,228,407.0,199,84,113,2,27,.990,3.46
40,"LeMahieu, D",NYY,2B,602,109,197,33,2,26,102,46,90,5,2,.327,.375,.518,0.893,0,2,1,4,312,61,14,184,136,2463,655,1241.2,620,351,261,8,63,.987,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,"Saladino, T",MIL,SS,65,7,8,0,0,2,8,5,26,2,0,.123,.197,.215,0.413,0,1,0,0,14,2,1,16,15,287,71,151.0,62,28,33,1,7,.984,2.10
688,"Barreto, F",OAK,2B,57,6,7,2,0,2,5,1,23,1,0,.123,.138,.263,0.401,0,0,0,0,15,4,0,18,9,237,58,146.2,71,20,46,5,8,.930,3.00
711,"Pedroia, D",BOS,2B,20,1,2,0,0,0,1,1,2,0,0,.100,.143,.100,0.243,0,0,0,0,2,0,2,9,7,77,21,28.0,16,9,7,0,3,1.000,2.67
727,"Velazquez, A",CLE,3B,23,3,2,2,0,0,0,1,13,1,0,.087,.125,.174,0.299,0,0,0,0,4,2,0,4,4,111,24,55.0,28,13,14,1,3,.964,2.25


In [11]:
# Playe와 Team 컬럼 삭제

df_total_infld.drop(['Player', 'Team'], axis=1, inplace=True)
df_total_infld.drop(['AB', 'INN'], axis=1, inplace=True)
df_total_infld.drop(['AVG', 'OBP', 'SLG', 'FPCT'], axis=1, inplace=True)

In [12]:
# 포지션 별로 각 수치 평균

df_total_infld.groupby('Pos').agg(['mean'])

,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,OPS,IBB,HBP,SAC,SF,TB,XBH,GDP,GO,AO,NP,PA,TC,PO,A,E,DP,RF
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
Pos,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1B,43.692308,75.846154,15.892308,0.800000,15.000000,47.707692,37.061538,81.338462,1.323077,0.584615,0.740569,1.938462,3.815385,0.030769,2.338462,138.338462,31.692308,7.338462,72.000000,77.738462,1415.000000,347.969231,532.092308,479.738462,48.061538,4.292308,47.646154,5.406923
2B,37.118421,70.500000,13.434211,1.552632,8.947368,33.078947,23.144737,60.171053,4.500000,1.789474,0.694263,0.934211,3.197368,0.815789,1.763158,113.881579,23.934211,5.118421,72.578947,72.052632,1164.934211,301.776316,271.144737,117.710526,147.868421,5.565789,35.802632,2.968684
3B,42.810811,77.324324,16.256757,1.378378,13.013514,42.959459,29.445946,71.486486,2.716216,1.094595,0.718946,1.378378,3.648649,0.324324,2.270270,135.378378,30.648649,6.500000,70.945946,78.337838,1307.081081,331.243243,225.743243,95.445946,123.500000,6.797297,18.081081,2.554054
SS,46.034483,85.879310,17.758621,1.982759,11.741379,41.103448,24.982759,71.362069,6.706897,2.396552,0.709328,1.155172,2.413793,0.827586,2.258621,142.827586,31.482759,6.982759,84.500000,82.672414,1348.086207,351.810345,311.517241,103.655172,199.810345,8.051724,42.155172,3.162414


In [13]:
df_total_infld.drop(['OPS', 'RF'], axis=1, inplace=True)
df_total_infld.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 22 to 751
Data columns (total 27 columns):
Pos    273 non-null object
R      273 non-null int64
H      273 non-null int64
2B     273 non-null int64
3B     273 non-null int64
HR     273 non-null int64
RBI    273 non-null int64
BB     273 non-null int64
SO     273 non-null int64
SB     273 non-null int64
CS     273 non-null int64
IBB    273 non-null int64
HBP    273 non-null int64
SAC    273 non-null int64
SF     273 non-null int64
TB     273 non-null int64
XBH    273 non-null int64
GDP    273 non-null int64
GO     273 non-null int64
AO     273 non-null int64
NP     273 non-null int64
PA     273 non-null int64
TC     273 non-null int64
PO     273 non-null int64
A      273 non-null int64
E      273 non-null int64
DP     273 non-null int64
dtypes: int64(26), object(1)
memory usage: 69.7+ KB


In [14]:
# label 데이터 레이블 인코딩
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_total_infld['Pos'])
df_total_infld['Pos'] = encoder.transform(df_total_infld['Pos'])
#labels = labels.reshape(-1, 1)
df_total_infld

,Pos,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,IBB,HBP,SAC,SF,TB,XBH,GDP,GO,AO,NP,PA,TC,PO,A,E,DP
22,2,61,115,23,1,17,62,27,49,2,1,1,4,0,5,191,41,11,93,82,1397,370,401,308,91,2,35
23,3,81,167,32,0,18,56,15,109,17,5,0,3,0,2,253,50,12,131,93,1765,518,532,182,324,26,75
24,1,54,109,20,1,4,28,36,29,2,2,1,1,0,3,143,25,2,86,105,1486,366,227,101,121,5,31
37,3,27,71,13,1,4,23,10,49,0,1,0,1,0,2,98,18,4,43,54,903,228,199,84,113,2,27
40,1,109,197,33,2,26,102,46,90,5,2,0,2,1,4,312,61,14,184,136,2463,655,620,351,261,8,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,3,7,8,0,0,2,8,5,26,2,0,0,1,0,0,14,2,1,16,15,287,71,62,28,33,1,7
688,1,6,7,2,0,2,5,1,23,1,0,0,0,0,0,15,4,0,18,9,237,58,71,20,46,5,8
711,1,1,2,0,0,0,1,1,2,0,0,0,0,0,0,2,0,2,9,7,77,21,16,9,7,0,3
727,2,3,2,2,0,0,0,1,13,1,0,0,0,0,0,4,2,0,4,4,111,24,28,13,14,1,3


In [15]:
# 데이터 세트 분리 (피처 데이터 and 라벨 데이터)

y_infield_df = df_total_infld['Pos']
x_infield_df = df_total_infld.drop('Pos', axis=1)

In [16]:
# train_test_split()를 통해 train과 test 데이터 분리
# from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_infield_df, y_infield_df, test_size=0.2, random_state=11)

In [17]:
print(x_infield_df.shape)
print(y_infield_df.shape)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(273, 26)
(273,)
(218, 26)
(218,)
(55, 26)
(55,)


# 1) 테스트로 여러가지 모델로 학습하고 예측/평가해보자
- Random Forest
- Decision Tree
- Gradient Boosting

In [38]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import GradientBoostingClassifier
deci_tree = DecisionTreeClassifier(random_state=11)
rand_fore = RandomForestClassifier(random_state=11)
grad_boos = GradientBoostingClassifier(random_state=11)
xgb_wrap = XGBClassifier(n_estimators=500, learning_rate=0.03, max_depth=3)

# Decision Tree
deci_tree.fit(x_train, y_train)
deci_tree_pred = deci_tree.predict(x_test)
print('결정트리 정확도: {0:.3f}'.format(accuracy_score(y_test, deci_tree_pred)))

# Random Forest
rand_fore.fit(x_train, y_train)
rand_fore_pred = rand_fore.predict(x_test)
print('랜덤포레스트의 정확도: {0:.3f}'.format(accuracy_score(y_test, rand_fore_pred)))

# GradientBoosting
grad_boos.fit(x_train, y_train)
grad_boos_pred = grad_boos.predict(x_test)
print('GBM의 정확도: {0:.3f}'.format(accuracy_score(y_test, grad_boos_pred)))

# XGBClassifier
xgb_wrap.fit(x_train, y_train)
xgb_wrap_preds = xgb_wrap.predict(x_test)
print('XGB의 정확도: {0:.3f}'.format(accuracy_score(y_test, xgb_wrap_preds)))

결정트리 정확도: 0.491
랜덤포레스트의 정확도: 0.545
GBM의 정확도: 0.600
XGB의 정확도: 0.655


### 교차검증 : k폴드개로 나눠서 돌아가면서 검증
- cross_val_score()사용

## 1) 결정트리

In [203]:
# from sklearn.model_selection import cross_val_score

scores = cross_val_score(deci_tree, x_infield_df, y_infield_df, cv=4)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6377
교차 검증 1 정확도: 0.6618
교차 검증 2 정확도: 0.5294
교차 검증 3 정확도: 0.4118
교차 검증 평균 정확도 : 0.5602


### 하이퍼파라미터 변화를 통한 트리 깊이 조절
- GridSearchCV : 하이퍼파라미터 조절

In [204]:
# from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [4, 6, 8, 10, 12],
    'min_samples_split': [2,4,6,8,10,12]
}

grid_cv = GridSearchCV(deci_tree, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


GridSearchCV 최고 평균 정확도 수치:0.5829
GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 8, 'min_samples_split': 6}


[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed:    0.8s finished


In [205]:
best_deci_tree = grid_cv.best_estimator_
pred1 = best_deci_tree.predict(x_test)
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

결정 트리 예측 정확도:0.5818


## 2) 랜덤포레스트

In [206]:
scores = cross_val_score(rand_fore, x_infield_df, y_infield_df, cv=3)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6703
교차 검증 1 정확도: 0.6484
교차 검증 2 정확도: 0.4505
교차 검증 평균 정확도 : 0.5897


In [208]:
# from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [10, 12, 14],
    'min_samples_split': [2,4,6,8,10,12]
    #'min_samples_leaf' : [2,4,6,8,10],
    #'n_estimators' : [10]
}

grid_cv = GridSearchCV(rand_fore, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:   15.6s finished


GridSearchCV 최고 평균 정확도 수치:0.5965
GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 12, 'min_samples_split': 4}


In [209]:
best_rand_fore = grid_cv.best_estimator_
pred1 = best_deci_tree.predict(x_test)
accuracy = accuracy_score(y_test, pred1)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

결정 트리 예측 정확도:0.5818


In [210]:
scores = cross_val_score(best_rand_fore, x_infield_df, y_infield_df, cv=3)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6264
교차 검증 1 정확도: 0.6813
교차 검증 2 정확도: 0.4725
교차 검증 평균 정확도 : 0.5934


## 3) GBM

In [44]:
scores = cross_val_score(grad_boos, x_infield_df, y_infield_df, cv=4)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6522
교차 검증 1 정확도: 0.6471
교차 검증 2 정확도: 0.5294
교차 검증 3 정확도: 0.5147
교차 검증 평균 정확도 : 0.5858


In [46]:
# from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[5, 10, 20, 30, 40, 50, 60, 70],
    'learning_rate':[0.01, 0.05, 0.06, 0.08],
}

grid_cv_g = GridSearchCV(grad_boos, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv_g.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv_g.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv_g.best_params_)

Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   30.2s finished


GridSearchCV 최고 평균 정확도 수치:0.6104
GridSearchCV 최적 하이퍼 파라미터: {'learning_rate': 0.06, 'n_estimators': 50}


In [47]:
best_grad_boos = grid_cv_g.best_estimator_
print(grid_cv_g.best_estimator_)
pred_g = best_grad_boos.predict(x_test)
accuracy = accuracy_score(y_test, pred_g)
print('결정 트리 예측 정확도:{0:.4f}'.format(accuracy))

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.06, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=50,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=11, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
결정 트리 예측 정확도:0.5273


In [48]:
scores = cross_val_score(best_grad_boos, x_infield_df, y_infield_df, cv=3)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6703
교차 검증 1 정확도: 0.6374
교차 검증 2 정확도: 0.4066
교차 검증 평균 정확도 : 0.5714


d
d

In [43]:
# from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb_wrap, x_infield_df, y_infield_df, cv=5)
for iter, accuracy in enumerate(scores):
    print('교차 검증 {0} 정확도: {1:.4f}'.format(iter, accuracy))

print('교차 검증 평균 정확도 : {0:.4f}'.format(np.mean(scores)))

교차 검증 0 정확도: 0.6909
교차 검증 1 정확도: 0.7818
교차 검증 2 정확도: 0.6000
교차 검증 3 정확도: 0.5556
교차 검증 4 정확도: 0.3889
교차 검증 평균 정확도 : 0.6034


In [49]:
# from sklearn.model_selection import GridSearchCV

params = {
    'max_depth' : [10, 12, 14],
    'min_samples_split': [2,4,6,8,10,12]
    #'min_samples_leaf' : [2,4,6,8,10],
    #'n_estimators' : [10]
}

grid_cv_x = GridSearchCV(xgb_wrap, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv_x.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv_x.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv_x.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  54 out of  54 | elapsed:  1.6min finished


GridSearchCV 최고 평균 정확도 수치:0.5962
GridSearchCV 최적 하이퍼 파라미터: {'max_depth': 12, 'min_samples_split': 2}


In [ ]:
xgb_wrap = GridSearchCV(grad_boos, param_grid=params, scoring='accuracy', cv=3, verbose=1)
grid_cv_g.fit(x_train, y_train)
print('GridSearchCV 최고 평균 정확도 수치:{0:.4f}'.format(grid_cv_g.best_score_))
print('GridSearchCV 최적 하이퍼 파라미터:', grid_cv_g.best_params_)

- Random Forest 모델로 GridSearchCV의 best 파라미터를 찾고 그 파라미터로 교차검증을 하니 테스트 정확도가 **54.5% > 59.34%** 증가

# 2) 테스트로 딥러닝으로 학습하고 예측/평가해보자
- DNN

In [50]:
x_train, x_test, y_train, y_test = train_test_split(x_infield_df, y_infield_df, test_size=0.2, random_state=11)

In [51]:
print(x_infield_df.shape)
print(y_infield_df.shape)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(273, 26)
(273,)
(218, 26)
(218,)
(55, 26)
(55,)


In [67]:
import keras
from keras import models
from keras import layers

### 학습할 때 data의 편차가 커서 정규화 해준다.

In [65]:
mean = x_train.mean(axis=0)
x_train -= mean
std = x_train.std(axis=0)
x_train /= std

mean = x_test.mean(axis=0)
x_test -= mean
std = x_test.std(axis=0)
x_test /= std

In [68]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=x_train.shape))
    model.add(layers.Dense(32, activation='relu'))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    model.summary()
    
    return model